In [6]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# from pandas.plotting import scatter_matrix
#import seaborn as sns
#import matplotlib as mpl
#from matplotlib import cm
import os
import sys
import datetime
from datetime import timedelta
import random

warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
# from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,f1_score, recall_score
from scipy.stats import uniform, randint

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import joblib #https://joblib.readthedocs.io/en/latest/
from imblearn.under_sampling import TomekLinks
from sklearn.neighbors import NearestNeighbors

# Generate and plot a synthetic imbalanced classification dataset
from collections import Counter
from imblearn.under_sampling import EditedNearestNeighbours

path = './'
# path = '/content/drive/MyDrive/Colab Notebooks/Industria_4/Analitica_Datos/'
name = 'household_power_consumption.csv'

## Dataset loading and preprocessing
1. During data loading the dates and times are parsed on an unique column using parse_dates parameter on read_csv function, the ? data is replaced with nan.

2. nan values are replaced by the corresponding column median.

3. Asign cathegorical tag tothe samples of the dataset. this is necessary beacuse subsequent steps involve the training of clasification models that identifies the kind of energy consumption corresponding to a global electrical measurements sample. Namely:
    * no consumption on any zone (___tag 0___)
    * zone 2 has the main consumption (___tag 1___)
    * zone 3 has the main consumption (___tag 2___)
    * the two zones has the same consumption (___tag 3___)
    
    zone 2 Corresponds to the Laundry room, containing washing machine, tumble dryer a refrigerator and a light.
    
    zone 3 corresponds to an electric water heater and an air conditioner.

Notes:
* ___Could be necessary to create more classes in order to add relevance to the classification task___

In [7]:
def load_preprocess_data():
    '''
    Read the csv file with ';' separator beetwen columns, parsing dates 
    and asigning consumption category tags to each sample, returns global 
    electrical varibles as X and cathegories as Y
    '''
    #Takes the columns 'Date' and 'Time' from the original csv and combines it on a single 'Date' Column 
    df = pd.read_csv(path+name, sep = ';',
                parse_dates={'TimeStamp':['Date','Time']},
                infer_datetime_format=True,
                low_memory=False, na_values=['nan','?']
                )

    df.sort_values(by='TimeStamp')
    
    #Data imputation replacing nan fields with the column median
    for i in df.columns[1:]:
        median = df[i].median() # option 3
        df[i].fillna(median, inplace=True)
    
    #select just the columns needed
    df3 = df.loc[:int(len(df)), ['Global_active_power', 'Global_reactive_power', 
                               'Voltage',	'Global_intensity',	'Sub_metering_2',
                               'Sub_metering_3']]
    df3['Global_apparent_power'] = (df['Global_active_power']**2+df['Global_reactive_power']**2)**(1/2)
    df3['Power_factor'] = df['Global_active_power']/df3['Global_apparent_power']
    
    #assign categorical tags to the data using the folowing rules, 
     
    conditions = [
    (df3['Sub_metering_2'] == 0) & (df3['Sub_metering_3'] == 0), ## Etiqueta 0: No hay consumo
    df3['Sub_metering_2'] > df3['Sub_metering_3'], ## Etiqueta 1: Mayor consumo Zona 2
    df3['Sub_metering_2'] < df3['Sub_metering_3'], ## Etiqueta 2: Mayor consumo Zona 3
    df3['Sub_metering_2'] == df3['Sub_metering_3']] ## Etiqueta 3: Consumo en ambas zonas por igual
    choices = [0, 1, 2, 3]
    y = np.select(conditions, choices, default=4)
    df3.drop(columns=['Sub_metering_2',	'Sub_metering_3'], inplace=True)

    counter = Counter(y)
    print('Distribución de las clases:', counter)

    df3['label'] = y
    Xdata = df3.loc[:int(len(df)), ['Global_active_power', 'Global_reactive_power', 
                               'Voltage','Global_intensity','Global_apparent_power','Power_factor']]

    return Xdata, y

## Split and Balance the Dataset
Due to the considerable high diferences in the quantity of samples for the various categories, it is necessary to make the training subset more balanced by means of undersampling the mayority classes. This is achieved using two undersampling steps first with EditedNearestNeighbours and then with TomekLinks.

Notes:
* ___could be interesting to have a range when zones are considered equal___
* ___Is needed to check performance using another undersampling or oversampling methods such as ADASYN or random___

In [8]:
def split_balance_data(Xdata, y):
    # Tamaño Xtrain 70%, Tamaño Xtest 15%, Xprod 15%
    Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, y, test_size=0.3, random_state=2)
    Xprod, Xtest, yprod, ytest = train_test_split(Xtest, ytest, test_size=0.5, random_state=2)
    Xtest1 = Xtest.copy()
    print('-Train (70%):', Xtrain.shape, '\n-Test (15%):', Xtest.shape, '\n-Production (15%):', Xprod.shape)
    
    # summarize class distribution
    counter = Counter(ytrain)
    print('Sin balancear las clases:', counter)

    # configure the undersampling
    undersample = EditedNearestNeighbours(n_neighbors=7, sampling_strategy = 'not minority', kind_sel='all')
    # transform the dataset
    X1, y1 = undersample.fit_resample(Xtrain, ytrain)
    # summarize the new class distribution
    counter = Counter(y1)
    print('Balanceo con Edited:', counter)

    # define the undersampling method
    under = TomekLinks()
    X2, y2 = under.fit_resample(X1, y1)
    # summarize the new class distribution
    counter = Counter(y2)
    print('Balanceo con TomeLinks:', counter)
    return X2, Xtest, Xprod, y2, ytest, yprod

In [9]:
#Here is executed the data loading, and pre-processing.
Xdata, y = load_preprocess_data()
X2, Xtest, Xprod, y2, ytest, yprod = split_balance_data(Xdata, y)

Distribución de las clases: Counter({2: 1040361, 0: 620104, 1: 314360, 3: 100434})
-Train (70%): (1452681, 6) 
-Test (15%): (311289, 6) 
-Production (15%): (311289, 6)
Sin balancear las clases: Counter({2: 728467, 0: 434376, 1: 219438, 3: 70400})
Balanceo con Edited: Counter({2: 213125, 3: 70400, 0: 63758, 1: 3386})
Balanceo con TomeLinks: Counter({2: 212703, 3: 69555, 0: 63426, 1: 3386})


In [10]:
X2.head(10)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Global_apparent_power,Power_factor
0,0.224,0.000,244.88,1.0,0.224000,1.000000
1,0.218,0.106,233.84,1.0,0.242405,0.899323
2,0.328,0.100,244.50,1.4,0.342905,0.956532
3,0.226,0.000,243.98,1.0,0.226000,1.000000
4,0.122,0.000,232.74,0.6,0.122000,1.000000
5,0.102,0.000,237.23,0.4,0.102000,1.000000
6,0.324,0.112,247.47,1.4,0.342812,0.945125
7,0.222,0.000,250.32,0.8,0.222000,1.000000
8,0.310,0.112,239.10,1.4,0.329612,0.940500
9,0.222,0.000,245.13,0.8,0.222000,1.000000


In [11]:
from sklearn.utils import class_weight

# Obtener los pesos de clase inversamente proporcionales a la frecuencia de clases
# class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y2), y=y2)
# , class_weight=dict(enumerate(class_weights))

## Pipeline
steps=[   
       [('std', StandardScaler()),
        # ('rep',  PCA()),
        ('cla', xgb.XGBClassifier(random_state=42))], # objective="multi:softprob", , eval_metric="auc"
      ]

# parameters = [{
#             #  'rep__n_components' : [0.99],
#              "cla__colsample_bytree": [0.3],
#               # "cla__gamma": [0.5],
#               "cla__learning_rate": [0.1], # default 0.1 
#               "cla__max_depth": [5], # default 3
#               "cla__n_estimators": [200], # default 100
#               "cla__subsample": [0.2],
#               "cla__scale_pos_weight": [3.8],
#               "cla__min_child_weight": [10],
#              }
#              ]

parameters = [
              {
              #"rep__n_components" : [0.99],
              "cla__colsample_bytree": [0.3, 0.4, 0.6, 0.7],
              "cla__gamma": [0, 0.5],
              "cla__learning_rate": [0.03, 0.05, 0.1, 0.2, 0.3 ], # default 0.1 
              "cla__max_depth": [2, 3, 4, 5], # default 3
              "cla__n_estimators": [10, 30, 100, 200], # default 100
              "cla__subsample": [0.4, 0.5, 0.6],
              "cla__min_child_weight": [10]
             }
             ]

#label_models = ['PCA_RandomForest_p3_','PCA_KNN_p3_', 'PCA_SVC_p3_']
label_models = ['XGB']

best_model = []
best_params = []
best_score = []

filename = path+'Results/Model_p3_other2'

for i in range(len(steps)):
    print('modelo %d/%d' % (i+1,len(steps)))
    grid_search = GridSearchCV(Pipeline(steps[i]), parameters[i], n_jobs=-1, cv=5,
                                scoring='balanced_accuracy', verbose=2)
    grid_search.fit(X2, y2)
    #print(grid_search.cv_results_)
    #mejor modelo entrenado
    best_model += [grid_search.best_estimator_]
    best_params += [grid_search.best_params_]
    best_score += [grid_search.best_score_]
    joblib.dump(best_model,filename+".pkl")

modelo 1/1
Fitting 5 folds for each of 1920 candidates, totalling 9600 fits


[CV] END cla__colsample_bytree=0.3, cla__gamma=0, cla__learning_rate=0.03, cla__max_depth=2, cla__min_child_weight=10, cla__n_estimators=10, cla__subsample=0.4; total time=   1.5s
[CV] END cla__colsample_bytree=0.3, cla__gamma=0, cla__learning_rate=0.03, cla__max_depth=2, cla__min_child_weight=10, cla__n_estimators=10, cla__subsample=0.5; total time=   1.5s
[CV] END cla__colsample_bytree=0.3, cla__gamma=0, cla__learning_rate=0.03, cla__max_depth=2, cla__min_child_weight=10, cla__n_estimators=10, cla__subsample=0.5; total time=   1.5s
[CV] END cla__colsample_bytree=0.3, cla__gamma=0, cla__learning_rate=0.03, cla__max_depth=2, cla__min_child_weight=10, cla__n_estimators=10, cla__subsample=0.4; total time=   1.6s
[CV] END cla__colsample_bytree=0.3, cla__gamma=0, cla__learning_rate=0.03, cla__max_depth=2, cla__min_child_weight=10, cla__n_estimators=10, cla__subsample=0.4; total time=   1.6s
[CV] END cla__colsample_bytree=0.3, cla__gamma=0, cla__learning_rate=0.03, cla__max_depth=2, cla__mi

In [12]:
print('Mejor modelo:\n')
print(best_model)
print(best_params)
print(best_score)
print(type(best_params))
print(best_params[0]['cla__colsample_bytree'])
params = best_params[0]

Mejor modelo:

[Pipeline(steps=[('std', StandardScaler()),
                ('cla',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.7, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.3,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=10, missing=nan,
                               monotone_constraints=None, multi_strategy=None,


In [13]:
# #max_depth
# maxd = 2
# #n_estimators
# ne = 200
# #subsample
# sb = 0.4
# #gamma 
# g = 0

model = xgb.XGBClassifier(objective="multi:softprob", random_state=42, eval_metric="auc",
                          colsample_bytree=params['cla__colsample_bytree'], 
                          gamma=params['cla__gamma'], learning_rate=params['cla__learning_rate'], 
                          max_depth=params['cla__max_depth'], min_child_weight= params['cla__min_child_weight'], 
                          n_estimators=params['cla__n_estimators'], subsample=params['cla__subsample'])

#train the model
sca = StandardScaler()
X2 = sca.fit_transform(X2)
# model = KNeighborsClassifier(n_neighbors=nneighb)
# model = xgb.XGBClassifier(objective="multi:softprob", random_state=42, eval_metric="auc", 
# max_depth=maxd, n_estimators=ne, subsample=sb, gamma=g)
model.fit(X2,y2)

#test the model
Xtest = sca.transform(Xtest)
ypred = model.predict(Xtest)
acc = accuracy_score(ytest, ypred)
rec = recall_score(ytest, ypred, average='weighted')
f1 = f1_score(ytest, ypred, average='weighted')

# print("Gamma: %s" % g)

# print("n_estimators: %s" % ne)

# print("max_depth: %s" % maxd)

# print("subsample: %s" % sb)


#show and results
print("  Accuracy: %s" % acc)
print("  F1-Score: %s" % f1)
print("  Recall: %s" % rec)

  Accuracy: 0.5251807805608293
  F1-Score: 0.555155672909072
  Recall: 0.5251807805608293
